# Understanding my finances

## Purpose
The purpose of this notebook is to expose some insights around my finances and where my money goes (expenditures).

We will consider two sources of data for this analysis: my chequing account statement and my credit card statement.

## Context and Scope
To inform interpretation of the data, some context and scope:  
  
- Salary is paid into the chequing account
- Chequing account transactions will include transfers between other accounts. These transactions should be excluded from this analysis, as our purpose is to understand my expenditures
- Credit card transactions will include payments of the balance, recorded as credits. These will be excluded from this analysis as they do not provide additional insight into my finances

In [ ]:
# Set up external imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import calendar
from scipy.stats import shapiro
import statsmodels.api as sm
idx = pd.IndexSlice

print("External modules imported")

In [ ]:
%matplotlib inline

In [ ]:
# Define file path
fpath = "./Personal_Finances/"

### Functions used in this notebook

In [ ]:
# Function to save plots to .png files
def generate_png(name):
    pltfile = fpath + name
    plt.savefig(pltfile, dpi=300, format="png")

In [ ]:
# Function to add a transaction type field
def add_transaction_type(col):
    if col < 0:
        return "D"
    else:
        return "C"

In [ ]:
print("Set up complete")

## 1. First, let us load the transaction data and do some basic formatting
__Raw data__:  

cheque_data  
credit_data  

__Formatted data__:  

tran_cheque_data  
tran_credit_data  

In [ ]:
# Find files
import os

for root, dirs, files in os.walk(fpath):
    for file in files:
        print(os.path.join(root, file))

In [ ]:
# Load chequing account transactions
cheque_data = pd.read_csv('.\Personal_Finances\chq.csv', skiprows=5)

In [ ]:
cheque_data.size

In [ ]:
# Get chequing data dtypes
cheque_data.dtypes

In [ ]:
# Format chequing data types
tran_cheque_data = cheque_data.drop(["Cheque Number","Date"], axis=1)
tran_cheque_data["TransactionDate"] = pd.to_datetime(cheque_data["Date"], format="%Y/%m/%d")
tran_cheque_data["Type"] = cheque_data["Amount"].apply(add_transaction_type)
tran_cheque_data["Amount"] = cheque_data["Amount"].apply(lambda x: abs(x))
tran_cheque_data["Raw Amount"] = cheque_data["Amount"]
# tran_cheque_data["Year"] = tran_cheque_data["TransactionDate"].dt.year
tran_cheque_data["Details"] = cheque_data[["Payee","Memo"]].fillna(value='').astype(str).agg(" ".join, axis=1)
tran_cheque_data = tran_cheque_data.assign(Account="CHQ")

In [ ]:
# Check formatted data
tran_cheque_data.head()

In [ ]:
# Load credit card transactions
credit_data = pd.read_csv('.\Personal_Finances\crd.csv')

In [ ]:
credit_data.dtypes

In [ ]:
credit_data.head()

In [ ]:
tran_credit_data = credit_data.drop(["Card"], axis=1)
tran_credit_data["TransactionDate"] = pd.to_datetime(credit_data["TransactionDate"], format="%d/%m/%Y")
tran_credit_data["ProcessedDate"] = pd.to_datetime(credit_data["ProcessedDate"], format="%d/%m/%Y")
tran_credit_data["ForeignTransaction"] = tran_credit_data["ForeignCurrencyAmount"].notnull()
# tran_credit_data["Year"] = tran_credit_data["TransactionDate"].dt.year
tran_credit_data = tran_credit_data.assign(Account="CRD")

In [ ]:
# Check formatted data
tran_credit_data.head()

## 2. Next, we will separate the transactions into credits and debits for each group.
We can expect the following broad groups of transactions:

For the chequing transactions:
  
Credits:
- Salary
- General credits
  
Debits:
- Transfers to other bank account (where we have the credit card)
- General debits

For the credit card transactions:
  
Credits:
- Paying off card balance
- Refunds
  
Debits:
- General transactions

In [ ]:
# Chequing data sets
chq_d = tran_cheque_data[tran_cheque_data["Type"] == "D"]
chq_c = tran_cheque_data[tran_cheque_data["Type"] == "C"]

In [ ]:
print("Chequing data count:", len(tran_cheque_data.index))
print("Chequing debits count:", len(chq_d.index))
print("Chequing credits count:", len(chq_c.index))

In [ ]:
# Credit card data sets
crd_d = tran_credit_data[tran_credit_data["Type"] == "D"]
crd_c = tran_credit_data[tran_credit_data["Type"] == "C"]

In [ ]:
print("Credit Card data shape:", len(tran_credit_data.index))
print("Credit Card debits shape:", len(crd_d.index))
print("Credit Card credits shape:", len(crd_c.index))

## 3. We want a unique set of debits and credits
We will now transform the data according to the scope highlighted at the start of this notebook.

We want to join the chequing and credit card debit sets to get a view of where our money is going, without duplicates.
We want to also join the chequing and credit card credits, while separating credit card payments.

We will separate transactions out of scope and store them in separate sets.

Finally, we want to apply the same format to the credit and debit data sets and append where appropriate for these sets:
  
1. Unique set of debits showing cash flow out of our accounts
2. Unique set of credits showing cash flow into our accounts
3. Credit card payments
4. Out of scope transactions for our accounts

### Let's have a look at the chequing account transactions to see if we can find some way to separate out of scope transactions

In [ ]:
## Identify what the Tran Type field corresponds to

# chq_d_cnt = chq_d.groupby(["Tran Type", "Year"]).size().sort_values(ascending=False)
chq_d_cnt = chq_d.groupby(["Tran Type"]).size().sort_values(ascending=False)

In [ ]:
# Transformation rules to filter out of scope data
# Remove savings and investment transactions
chq_d_flt = # details omitted
len(chq_d_flt)

In [ ]:
chq_d

In [ ]:
## Get amount values by type
# chq_d_sum = chq_d.groupby(["Tran Type", "Year"])["Amount"].sum().sort_values(ascending=False)
chq_d_sum = chq_d.groupby(["Tran Type"])["Amount"].sum().sort_values(ascending=False)

In [ ]:
## Get summarised values
chq_d_sv = pd.DataFrame(chq_d_cnt).join(pd.DataFrame(chq_d_sum))
chq_d_sv

In [ ]:
# chq_c_cnt = chq_c.groupby(["Tran Type", "Year"]).size().sort_values(ascending=False)
chq_c_cnt = chq_c.groupby(["Tran Type"]).size().sort_values(ascending=False)

In [ ]:
## Get amount values by type
# chq_c_sum = chq_c.groupby(["Tran Type", "Year"])["Amount"].sum().sort_values(ascending=False)
chq_c_sum = chq_c.groupby(["Tran Type"])["Amount"].sum().sort_values(ascending=False)

In [ ]:
## Get summarised values
chq_c_sv = pd.DataFrame(chq_c_cnt).join(pd.DataFrame(chq_c_sum))
chq_c_sv

### All credit card credit transactions are essentially in scope. We need to check if debit transactions will need to be filtered

In [ ]:
# Visual inspection of credit transactions 
crd_c.head()

In [ ]:
crd_d.dtypes

### There are a few refund transactions. Nothing is available to filter these on, may affect our stats slightly

### Let's get our common set of debits

In [ ]:
# # Verify columns and get common set for debits
# crd_d.columns
# chq_d.columns

# cols = ['TransactionDate', 'Year', 'Account', 'Type', 'Details', 'Amount']
cols = ['TransactionDate', 'Account', 'Type', 'Details', 'Amount']
combined_d = crd_d[cols].append(chq_d_flt[cols])

In [ ]:
# Let's look at the 10 top transactions by Amount
combined_d.sort_values(by="Amount", ascending=False).head(10)

## 4. Aggregate visualisations on combined debit set

### Weekly aggregate view

In [ ]:
# tmp = combined_d.join(combined_d["TransactionDate"].dt.isocalendar().set_index(combined_d["TransactionDate"]).drop_duplicates(), on="TransactionDate").drop("day", axis=1).groupby(["Account", "year", "week"]).sum().unstack([1,0]) 
# tmp.columns = tmp.columns.droplevel(0)
# tmp = tmp.sort_index(axis=1)
# # tmp.columns = tmp.columns
# tmp == combined_d_week_by_acc

In [ ]:
# 1. aggregate our combined transactions to get the sums per week
combined_d_week = combined_d.groupby(["Account"]).resample("W", on="TransactionDate").sum()
combined_d_week.reset_index(0, inplace=True)
# 2. get week number and group by account, week to get our summarised stats
combined_d_week_by_acc = combined_d_week.join(combined_d_week.index.isocalendar().drop_duplicates()).pivot(index="week", columns=["year","Account"], values="Amount")

In [ ]:
combined_d_week_total = combined_d_week_by_acc.groupby("year", axis=1).sum()

In [ ]:
week_2020 = combined_d_week_by_acc.loc[:,idx[2020]]

for i in range(week_2020.shape[1]):
    plt.plot(week_2020.iloc[:,i], label = week_2020.columns[i])
    
plt.xlabel("Week of Year")
plt.ylabel("Amount $NZD")
plt.title("Spend vs Week of Year 2020")
plt.legend()
plt.tick_params(axis='y',
                which='both',
                left=False,
                labelleft=False)
# Adjust plot spacing
size = plt.gcf().get_size_inches()
size[0] *= 1.4
plt.gcf().set_size_inches(size)
# plt.ylim(top=n)

generate_png("spend_vs_week_of_year_2020.png")

In [ ]:
week_2019 = combined_d_week_by_acc.loc[:,idx[2019]]

for i in range(week_2019.shape[1]):
    plt.plot(week_2019.iloc[:,i], label = week_2019.columns[i])

plt.xlabel("Week of Year")
plt.ylabel("Amount $NZD")
plt.title("Spend vs Week of Year 2019")
plt.legend()
plt.tick_params(axis='y',
                which='both',
                left=False,
                labelleft=False)
# Adjust plot spacing
size = plt.gcf().get_size_inches()
size[0] *= 1.4
plt.gcf().set_size_inches(size)
# plt.ylim(top=n)

generate_png("spend_vs_week_of_year_2019.png")

In [ ]:
for i in range(combined_d_week_total.shape[1]):
    plt.plot(combined_d_week_total.iloc[:,i], label = combined_d_week_total.columns[i])
    
expected = pd.Series(n, index=week_2019.index)
plt.plot(expected, label='expected')

combined_d_week_total.mean()

for i in range(len(combined_d_week_total.mean())):
    plt.plot(pd.Series(combined_d_week_total.mean().iloc[i], index=combined_d_week_total.index), label = 'mean ' + str(combined_d_week_total.mean().index[i]))

# mean = pd.Series(combined_d_week_total.mean(), index=week_2019.index)
# plt.plot(mean, label='mean')

plt.xlabel("Week of Year")
plt.ylabel("Amount $NZD")
plt.title("Spend vs Week of Year")
plt.legend()
plt.tick_params(axis='y',
                which='both',
                left=False,
                labelleft=False)
# Adjust plot spacing
size = plt.gcf().get_size_inches()
size[0] *= 1.4
plt.gcf().set_size_inches(size)
# plt.ylim(top=n)

generate_png("spend_vs_week_of_year.png")

### Day of week aggregate view

In [ ]:
# Aggregate by day-of-week/account
# 1. aggregate our combined transactions to get the sums per date
combined_d_dow_by_acc = combined_d.groupby(["Account","TransactionDate"]).sum()
# 2. get our day of week
combined_d_dow_by_acc = combined_d_dow_by_acc.reset_index(0).join(pd.date_range('2019-01-01','2020-12-31').isocalendar(), on="TransactionDate").drop(["week","year"], axis=1)
# 3. group by account, day-of-week and get our summarised stats
combined_d_dow_by_acc = combined_d_dow_by_acc.groupby(["Account","day"]).agg([np.sum, np.mean, np.median, np.size]).unstack(0)
# 4. get day-of-week names
combined_d_dow_by_acc.set_index(combined_d_dow_by_acc.reset_index()["day"].apply(lambda x: calendar.day_name[x-1]), inplace=True)
# 5. remove column multi-index level
combined_d_dow_by_acc.columns = combined_d_dow_by_acc.columns.droplevel(0)

In [ ]:
# Aggregate by day-of-week
# 1. aggregate our combined transactions to get the sums per date
combined_d_dow_total = combined_d.groupby(["TransactionDate"]).sum()
# 2. get our day of week
combined_d_dow_total = combined_d_dow_total.reset_index(0).join(pd.date_range('2019-01-01','2020-12-31').isocalendar(), on="TransactionDate").drop(["week","year"], axis=1)
# 3. group by day-of-week and get our summarised stats
combined_d_dow_total = combined_d_dow_total.groupby(["day"]).agg([np.sum, np.mean, np.median, np.size])
# 4. get day-of-week names
combined_d_dow_total.set_index(combined_d_dow_total.reset_index()["day"].apply(lambda x: calendar.day_name[x-1]), inplace=True)
# 5. remove column multi-index level
combined_d_dow_total.columns = combined_d_dow_total.columns.droplevel(0)

In [ ]:
combined_d_dow_by_acc

In [ ]:
combined_d_dow_total

In [ ]:
mean_data = combined_d_dow_by_acc.loc[:,idx['mean']]
for i in range(mean_data.shape[1]):
    plt.plot(mean_data.iloc[:,i], label = mean_data.columns[i])
    
expected = pd.Series(n, index=mean_data.index)
plt.plot(expected, label='expected')

plt.plot('mean', data=combined_d_dow_total, label='combined')
plt.xlabel("Day of Week")
plt.ylabel("Amount $NZD")
plt.title("Mean Spend vs Day of Week")
plt.legend()
plt.tick_params(axis='y',
                which='both',
                left=False,
                labelleft=False)
# Adjust plot spacing
size = plt.gcf().get_size_inches()
size[0] *= 1.4
plt.gcf().set_size_inches(size)
# print(combined_d_dow_by_acc.loc[:,idx[['mean','size'], :]])
# print(combined_d_dow_total.loc[:,['mean','size']])

generate_png("mean_spend_vs_day_of_week.png")

In [ ]:
median_data = combined_d_dow_by_acc.loc[:,idx['median']]
for i in range(median_data.shape[1]):
    plt.plot(median_data.iloc[:,i], label = median_data.columns[i])
    
expected = pd.Series(n, index=median_data.index)
plt.plot(expected, label='expected')
    
# plt.plot('mean', data=combined_d_dow_by_acc.loc[:,idx[:, 'CHQ']], label='CHQ')
# plt.plot('mean', data=combined_d_dow_by_acc.loc[:,idx[:, 'CRD']], label='CRD')
plt.plot('median', data=combined_d_dow_total, label='Combined')
plt.xlabel("Day of Week")
plt.ylabel("Amount $NZD")
plt.title("Median Spend vs Day of Week")
plt.legend()
plt.tick_params(axis='y',
                which='both',
                left=False,
                labelleft=False)
# Adjust plot spacing
size = plt.gcf().get_size_inches()
size[0] *= 1.4
plt.gcf().set_size_inches(size)
# print(combined_d_dow_by_acc.loc[:,idx[['mean','size'], :]])
# print(combined_d_dow_total.loc[:,['mean','size']])

generate_png("median_spend_vs_day_of_week.png")

In [ ]:
# Transaction count by Account, day-of-week
# 1. get our day of week
combined_d_dow_count_by_acc = combined_d.join(pd.date_range('2019-01-01','2020-12-31').isocalendar(), on="TransactionDate").drop(["week","year"], axis=1)
# 2. group by account, day-of-week and get our count
combined_d_dow_count_by_acc = combined_d_dow_count_by_acc.groupby(["Account","day"]).size().unstack(0)
# 3. get day-of-week names
combined_d_dow_count_by_acc.set_index(combined_d_dow_count_by_acc.reset_index()["day"].apply(lambda x: calendar.day_name[x-1]), inplace=True)

In [ ]:
# Transaction count by day-of-week
# 1. get our day of week
combined_d_dow_count_total = combined_d.join(pd.date_range('2019-01-01','2020-12-31').isocalendar(), on="TransactionDate").drop(["week","year"], axis=1)
# 2. group by day-of-week and get our count
combined_d_dow_count_total = pd.DataFrame(combined_d_dow_count_total.groupby(["day"]).size().rename("count"))
# 3. get day-of-week names
combined_d_dow_count_total.set_index(combined_d_dow_count_total.reset_index()["day"].apply(lambda x: calendar.day_name[x-1]), inplace=True)

In [ ]:
size_data = combined_d_dow_count_by_acc
for i in range(size_data.shape[1]):
    plt.plot(size_data.iloc[:,i], label = size_data.columns[i])
    
plt.plot('count', data=combined_d_dow_count_total, label='Combined')

plt.xlabel("Day of Week")
plt.ylabel("Transaction Count")
plt.title("Number of Transactions vs Day of Week")
plt.legend()
plt.tick_params(axis='y',
                which='both',
                left=False,
                labelleft=False)
# Adjust plot spacing
size = plt.gcf().get_size_inches()
size[0] *= 1.4
plt.gcf().set_size_inches(size)
# print(combined_d_dow_by_acc.loc[:,idx[['mean','size'], :]])
# print(combined_d_dow_total.loc[:,['mean','size']])

generate_png("transactions_vs_day_of_week.png")

## Weekly spend distribution

In [ ]:
combined_d_week_total.unstack().max()

In [ ]:
combined_d_week_total_all = combined_d_week_total.unstack()

n, bins, patches = plt.hist(x=combined_d_week_total_all,
                            bins='auto',
                            alpha=0.7, 
                            rwidth=0.85)
plt.xlabel('Amount ($NZD)')
plt.ylabel('Frequency')
plt.title('Amount of Weekly Spend (n=106)')
plt.tick_params(axis='x',
                which='both',
                bottom=False,
                labelbottom=False)

generate_png("amount_of_weekly_spend.png")

In [ ]:
shapiro(combined_d_week_total_all)

In [ ]:
sm.qqplot(combined_d_week_total_all, line ='45')
plt.tick_params(axis='both',
                which='both',
                bottom=False,
                labelbottom=False,
                left=False,
                labelleft=False)

generate_png("amount_of_weekly_spend_qqplot.png")

In [ ]:
n, bins, patches = plt.hist(x=combined_d_week_total, 
                            bins='auto',
                            label=combined_d_week_total.columns,
                            alpha=0.7, 
                            rwidth=0.85)
plt.xlabel('Amount ($NZD)')
plt.ylabel('Frequency')
plt.title('Amount of Weekly Spend by year (n=106)')
plt.legend()
plt.tick_params(axis='x',
                which='both',
                bottom=False,
                labelbottom=False)

generate_png("amount_of_weekly_spend_by_year.png")

In [ ]:
shapiro(combined_d_week_total[2020])

In [ ]:
shapiro(combined_d_week_total[2019])

In [ ]:
sm.qqplot(combined_d_week_total[2020], line ='45')

In [ ]:
sm.qqplot(combined_d_week_total[2019], line ='45')

## Daily spend distribution

In [ ]:
combined_d_daily = combined_d.resample('D', on="TransactionDate").sum()

In [ ]:
n, bins, patches = plt.hist(x=combined_d_daily, 
                            bins=bins,
                            label=combined_d_daily.columns,
                            alpha=0.7, 
                            rwidth=0.85)
plt.xlabel('Amount ($NZD)')
plt.ylabel('Frequency')
plt.title('Amount of Daily Spend (n=731)')
plt.tick_params(axis='x',
                which='both',
                bottom=False,
                labelbottom=False)

generate_png("amount_of_daily_spend.png")

In [ ]:
n, bins, patches = plt.hist(x=combined_d_daily, 
                            bins='auto',
                            label=combined_d_daily.columns,
                            alpha=0.7, 
                            rwidth=0.85)
plt.xlabel('Amount ($NZD)')
plt.ylabel('Frequency')
plt.title('Amount of Daily Spend (n=731)')
plt.tick_params(axis='x',
                which='both',
                bottom=False,
                labelbottom=False)

generate_png("amount_of_daily_spend_full.png")

In [ ]:
sm.qqplot(combined_d_daily, line ='45')
plt.tick_params(axis='both',
                which='both',
                bottom=False,
                labelbottom=False,
                left=False,
                labelleft=False)

generate_png("amount_of_daily_spend_qqplot.png")

In [ ]:
combined_d_daily.plot(kind='box')

In [ ]:
combined_d.groupby('Account')["Amount"].sum()

In [ ]:
combined_d[combined_d["Account"]=="CHQ"].sort_values(by="Amount", ascending=False).head(15)